## Data cleaning with pandas

Pandas is a python library to easily work with data. It's ideal for data cleaning and tranforming offering a wide range of methods to replace, drop, slice and merge parts or entire dataframes. This tutorial will use the data generated through our scraping operation on Rotten Tomatoes. We will quickly see how to load data, consolidate their structure, change data format and create/export smaller datasets that later will be used for visualization purposes.

This is a quick pandas overview. We won't cover statistics or data analysis tasks together right away. You can find plenty of resources on the web that will help you with more delicate and time consuming tasks, this is just a way for you to get started.

In [13]:
import pandas as pd
import numpy as np
import datetime as dt
import re

#Disable warning after check its safety in: https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None  # default='warn'

#### Load and preview data

We can easily open a csv file by specifying its directory. If we use `pd.read_csv` we also automatically convert our file into a Dataframe that can be later manipulated. We can also display data very conveniently by using `.sample()`, `.head()` or `.tail()`.

In [89]:
#Reading files
allTitles = pd.read_csv('data/unclean/movie_length_to_rating.csv', sep=",")

# Print a sample of rows
allTitles.sample(5)

,Unnamed: 0,Title,Runtime,Release Date,IMDB Rating,Critics Score,Votes,Gross
169,169,The Shape of Water,123 min,(2017),7.3,87,400144,63859435
122,122,Baby Driver,113 min,(2017),7.6,86,492723,107825862
141,141,Spotlight,129 min,(I) (2015),8.1,93,450004,45055776
69,69,Parasite,132 min,(2019),8.6,96,700996,53367844
188,188,The Illusionist,110 min,(2006),7.6,68,365274,39868642


In [4]:
# Print the first 5 rows in the dataframe
allTitles.head(5)

,Unnamed: 0,Title,Runtime,Release Date,IMDB Rating,Critics Score
0,0,The Shawshank Redemption,142 min,(1994),9.3,80
1,1,The Dark Knight,152 min,(2008),9.0,84
2,2,Fight Club,139 min,(1999),8.8,66
3,3,Forrest Gump,142 min,(1994),8.8,82
4,4,Pulp Fiction,154 min,(1994),8.9,94


In [5]:
# Print the last 5 rows in the dataframe
allTitles.tail(5)

,Unnamed: 0,Title,Runtime,Release Date,IMDB Rating,Critics Score
245,245,The Secret Life of Walter Mitty,114 min,(2013),7.3,54
246,246,Lord of War,122 min,(2005),7.6,62
247,247,Gone with the Wind,238 min,(1939),8.1,97
248,248,Annihilation,115 min,(I) (2018),6.8,79
249,249,Magnolia,188 min,(1999),8.0,77


#### Change dataframe columns

We can drop, sort, create or rename columns very easily.

In [90]:
# Drop the index column that I accidentally exported when scraping 
#(it can be easily avoided by setting Index=False when exporting the csv)
allTitles = allTitles.drop(['Unnamed: 0'], axis=1)

In [91]:
# Print column names
allTitles.columns

Index(['Title', 'Runtime', 'Release Date', 'IMDB Rating', 'Critics Score',
       'Votes', 'Gross'],
      dtype='object')

In [92]:
allTitles.columns

Index(['Title', 'Runtime', 'Release Date', 'IMDB Rating', 'Critics Score',
       'Votes', 'Gross'],
      dtype='object')

#### Change data format

Another annoying task is changing data format. We have at our disposal a series of methods to do that, not only for standard formats such as strings or integers, but also for "special" ones like datetime.

In [8]:
# We can check our result by printing the unique values for a particular column.
allTitles['Release Date'].unique()

array(['(1994)', '(2008)', '(1999)', '(2001)', '(2003)', '(1972)',
       '(2014)', '(2012)', '(2002)', '(1995)', '(2000)', '(2005)',
       '(2009)', '(1991)', '(1998)', '(2013)', '(1993)', '(2006)',
       '(1974)', '(2019)', '(1997)', '(1990)', '(1992)', '(1975)',
       '(1980)', '(2004)', '(2007)', '(2011)', '(2015)', '(1983)',
       '(1976)', '(1957)', '(1982)', '(2010)', '(1987)', '(2017)',
       '(I) (2015)', '(1996)', '(II) (2016)', '(1979)', '(I) (2011)',
       '(1942)', '(2016)', '(2018)', '(1988)', '(I) (2013)', '(1989)',
       '(I) (2008)', '(1946)', '(I) (2004)', '(2021)', '(1941)', '(1986)',
       '(1984)', '(I) (2016)', '(1985)', '(I) (2010)', '(I) (2014)',
       '(I) (2012)', '(1954)', '(I) (2009)', '(1978)', '(I) (2007)',
       '(1962)', '(1939)', '(I) (2018)'], dtype=object)

In [93]:
#Removing one movie because of this incorrect field.
cleanedTitles = allTitles[allTitles['Release Date'] != '1h 27m']
cleanedTitles = cleanedTitles[cleanedTitles['Gross'] != 'none']

In [81]:
# We can check data format by using .dtypes. In pandas strings are usually displayed as objects.
allTitles.dtypes

Unnamed: 0         int64
Title             object
Runtime           object
Release Date      object
IMDB Rating      float64
Critics Score      int64
Votes              int64
Gross             object
dtype: object

In [95]:
# Convert again datetime obj into string to obtain only date (we use the dt method .date())
cleanedTitles['Release Date'] = cleanedTitles['Release Date'].map(lambda x: re.search(r'\d{4}', x).group(0))
cleanedTitles['Runtime'] = cleanedTitles['Runtime'].map(lambda x: re.search(r'\d{1,3}', x).group(0))
cleanedTitles['Critics Score'] = cleanedTitles['Critics Score'].map(lambda x: round(x * 0.1, 1))

In [96]:
# Check the output
cleanedTitles['Critics Score'].unique()

array([ 8. ,  8.4,  6.6,  8.2,  9.4,  9.2, 10. ,  7.4,  7.8,  8.7,  6.5,
        8.1,  6.7,  7. ,  6.9,  8.5,  9.1,  7.5,  6.1,  9. ,  5.9,  6.4,
        6.2,  7.7,  8.8,  6.8,  7.9,  8.9,  7.2,  5.7,  9.6,  5.2,  7.6,
        7.1,  9.5,  8.6,  8.3,  9.8,  7.3,  9.3,  5.3,  5.6,  5.5,  4.9,
        3. ,  5.8,  4.1,  6. ,  4.7,  4.8,  5.1,  6.3,  4.4,  3.3,  4.5,
        5. ,  4.6,  5.4,  9.7])

#### Split rows, adapt structure, create subselections and data slices

It is also possible to adapt the structure of our data by adding columns, splitting rows and slicing the dataframe to smaller ones. 

In [45]:
cleanedTitles['Runtime'].unique()

array(['142', '152', '139', '154', '178', '201', '175', '169', '164',
       '179', '127', '165', '155', '140', '153', '118', '180', '195',
       '130', '151', '189', '202', '122', '194', '110', '146', '132',
       '119', '111', '106', '103', '88', '96', '181', '99', '133', '108',
       '107', '149', '126', '141', '135', '120', '112', '144', '102',
       '91', '170', '150', '114', '113', '117', '156', '116', '101',
       '137', '134', '95', '93', '161', '147', '166', '100', '148', '143',
       '168', '138', '158', '128', '123', '162', '163', '105', '157',
       '90', '109', '115', '92', '129', '131', '125', '167', '187', '121',
       '160', '89', '97', '94', '136', '209', '172', '229', '207', '183',
       '124', '159', '238', '188'], dtype=object)

In [97]:
#Export top 100 selection
cleanedTitles.to_csv('data/movies_length_rating.csv', sep=",", index=False, na_rep='NaN')